In [1]:
#!/usr/bin/env python

# ROSpy node to subscribe to a ROS image topic
# Taken and modified from http://wiki.ros.org/rospy_tutorials/Tutorials/WritingImagePublisherSubscriber

from __future__ import division
#from __future__ import print_function

# Python libs
import sys, time

# numpy and scipy
import numpy as np
from scipy.ndimage import filters

from features_extraction import points_extraction

# OpenCV
import cv2

# Ros libraries
import roslib
import rospy
from rospy.numpy_msg import numpy_msg
from rospy_tutorials.msg import Floats

# Ros Messages
from sensor_msgs.msg import CompressedImage
# We do not use cv_bridge it does not support CompressedImage in python
# from cv_bridge import CvBridge, CvBridgeError

VERBOSE=False
SHOW_OPENCV=True

class image_processing:

    def __init__(self):
        
        # Topics where to publish
        self.image_pub = rospy.Publisher("/output/image_raw/compressed",
            CompressedImage)
        
        self.vis_feat = rospy.Publisher("/output/visual_features", numpy_msg(Floats), queue_size=10)

        # Subscribed topic
        self.subscriber = rospy.Subscriber("/camera/rgb/image_raw/compressed",
            CompressedImage, self.callback,  queue_size = 1)
        
        if VERBOSE :
            print "subscribed to /camera/rgb/image_raw/compressed"
            
        # Instantiate the point extraction class
        self.points_ex = points_extraction()
    
    def callback(self, ros_data):

        '''Callback function of subscribed topic.'''
        if VERBOSE :
            print 'received image of type: "%s"' % ros_data.format
                
        # Direct conversion to CV2
        np_arr = np.fromstring(ros_data.data, np.uint8)

        #image_np = cv2.imdecode(np_arr, cv2.CV_LOAD_IMAGE_COLOR)
        image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR) # OpenCV >= 3.0:
        
        # Run feature point features extraction algorithm
        self.points_ex.run(image)
        
        # Get the processed image
        image_proc = self.points_ex.get_image()
    
        # Show the image with openCV
        if SHOW_OPENCV:
            cv2.imshow('Robot camera', image_proc)
            
            key = cv2.waitKey(1)

            self.stop_it = False
            if key == ord('q'):
                self.stop_it = True
                cv2.destroyAllWindows()
                rospy.signal_shutdown('Quit')
            if key == ord('r'):
                self.points_ex.track = False
        
        # Create CompressedImage to be published
        msg = CompressedImage()
        msg.header.stamp = rospy.Time.now()
        msg.format = "jpeg"
        msg.data = np.array(cv2.imencode('.jpg', image_proc)[1]).tostring()
        
        # Publish the processed image
        self.image_pub.publish(msg)

        # Get and publish the visual features
        s = self.points_ex.get_features()
        s_msg = np.array(s, dtype=np.float32)
        self.vis_feat.publish(s_msg)

def main(args):

    '''Initializes and cleanup ros node'''
    
    rospy.init_node('image_processing', anonymous=True)
    
    ic = image_processing()
    
    try:
        rospy.spin()

    except KeyboardInterrupt:
        print "Shutting down ROS image_processing module"
    
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main(sys.argv)

11.19338989 236.33110046]
 [328.8420105  184.97911072]
 [243.94384766 152.00723267]]
a:  [211.19338989 236.33110046]
b:  [[243.94207764 152.00836182]
 [328.83129883 184.98303223]
 [296.20761108 269.30487061]
 [211.19462585 236.33273315]]
dist:  [9.04588348e+01 1.28356152e+02 9.11849073e+01 2.04774996e-03]
idx:  3
 
2 ----------
blobs:  [[296.21438599 269.30535889]
 [211.19338989 236.33110046]
 [328.8420105  184.97911072]
 [243.94384766 152.00723267]]
a:  [328.8420105  184.97911072]
b:  [[243.94207764 152.00836182]
 [328.83129883 184.98303223]
 [296.20761108 269.30487061]
 [211.19462585 236.33273315]]
dist:  [9.10772688e+01 1.14069323e-02 9.04203396e+01 1.28367058e+02]
idx:  1
 
3 ----------
blobs:  [[296.21438599 269.30535889]
 [211.19338989 236.33110046]
 [328.8420105  184.97911072]
 [243.94384766 152.00723267]]
a:  [243.94384766 152.00723267]
b:  [[243.94207764 152.00836182]
 [328.83129883 184.98303223]
 [296.20761108 269.30487061]
 [211.19462585 236.33273315]]
dist:  [2.09951179e-03

In [2]:
a = np.array([[1,0],[2,2],[4,1]])
b = np.array([[2,3],[3,1],[0,4]])
dist = np.linalg.norm(a[0]-a,2,0)
print('dist', dist)
#print(sort(dist))
idxs = np.argmin(dist)
print(idxs)
print(a)
print(a[idxs])

('dist', array([3.16227766, 2.23606798]))
1
[[1 0]
 [2 2]
 [4 1]]
[2 2]


In [3]:
a = np.array([np.pi,2,3,4])

print(np.abs(a-np.pi))
if np.any(np.abs(a-np.pi)<0.1):
    print('here')
    a = np.arcsin(np.sin(a))
 
print(a)

[0.         1.14159265 0.14159265 0.85840735]
here
[ 1.22464680e-16  1.14159265e+00  1.41592654e-01 -8.58407346e-01]
